In [1]:
import numpy as np
import Cluster_Ensembles as CE
from functools import reduce

In [2]:
# require(data.table)
# require(bit64)
# require(dbscan)
# require(doParallel)
# require(rBayesianOptimization)
# path='../input/train_1/'
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

from trackml.dataset import load_event, load_dataset
from trackml.score import score_event
from trackml.randomize import shuffle_hits

from sklearn.preprocessing import StandardScaler
import hdbscan as _hdbscan
from scipy import stats
from tqdm import tqdm

import time

from sklearn.cluster.dbscan_ import dbscan
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

from sklearn.neighbors import KDTree
import hdbscan
from bayes_opt import BayesianOptimization

# https://www.ellicium.com/python-multiprocessing-pool-process/
# http://sebastianraschka.com/Articles/2014_multiprocessing.html
from multiprocessing import Pool

In [3]:
import os
import time
import hdbscan as _hdbscan
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

In [4]:
def create_one_event_submission(event_id, hits, labels):
    sub_data = np.column_stack(([event_id]*len(hits), hits.hit_id.values, labels))
    submission = pd.DataFrame(data=sub_data, columns=["event_id", "hit_id", "track_id"]).astype(int)
    return submission

In [5]:
def preprocess(hits):        
    x = hits.x.values
    y = hits.y.values
    z = hits.z.values

    r = np.sqrt(x**2 + y**2 + z**2)
    hits['x2'] = x/r
    hits['y2'] = y/r

    r = np.sqrt(x**2 + y**2)
    hits['z2'] = z/r

    ss = StandardScaler()
    X = ss.fit_transform(hits[['x2', 'y2', 'z2']].values)
#     for i, rz_scale in enumerate(self.rz_scales):
#         X[:,i] = X[:,i] * rz_scale

    return X

In [6]:
def _eliminate_outliers(clusters,M):
    my_labels = np.unique(clusters)
    norms=np.zeros((len(my_labels)),np.float32)
    indices=np.zeros((len(my_labels)),np.float32)
    for i, cluster in tqdm(enumerate(my_labels),total=len(my_labels)):
        if cluster == 0:
            continue
        index = np.argwhere(clusters==cluster)
        index = np.reshape(index,(index.shape[0]))
        indices[i] = len(index)
        x = M[index]
        norms[i] = self._test_quadric(x)
    threshold1 = np.percentile(norms,90)*5
    threshold2 = 25
    threshold3 = 6
    for i, cluster in enumerate(my_labels):
        if norms[i] > threshold1 or indices[i] > threshold2 or indices[i] < threshold3:
            clusters[clusters==cluster]=0
            
def _test_quadric(x):
    if x.size == 0 or len(x.shape)<2:
        return 0
    Z = np.zeros((x.shape[0],10), np.float32)
    Z[:,0] = x[:,0]**2
    Z[:,1] = 2*x[:,0]*x[:,1]
    Z[:,2] = 2*x[:,0]*x[:,2]
    Z[:,3] = 2*x[:,0]
    Z[:,4] = x[:,1]**2
    Z[:,5] = 2*x[:,1]*x[:,2]
    Z[:,6] = 2*x[:,1]
    Z[:,7] = x[:,2]**2
    Z[:,8] = 2*x[:,2]
    Z[:,9] = 1
    v, s, t = np.linalg.svd(Z,full_matrices=False)        
    smallest_index = np.argmin(np.array(s))
    T = np.array(t)
    T = T[smallest_index,:]        
    norm = np.linalg.norm(np.dot(Z,T), ord=2)**2
    return norm

In [7]:
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

#------------------------------------------------------

def make_counts(labels):
    
    
    _,reverse,count = np.unique(labels,return_counts=True,return_inverse=True)
    counts = count[reverse]
    counts[labels==0]=0
    
    return counts

def one_loop(param):
    
    # <todo> tune your parameters or design your own features here! 
    
    i,m, x,y,z, d,r, a, a_start,a_step = param
    #print('\r %3d  %+0.8f '%(i,da), end='', flush=True)
    
    da = m*(a_start - (i*a_step))
    aa = a + np.sign(z)*z*da 
    zr = z/r
    
    X = StandardScaler().fit_transform(np.column_stack([aa, aa/zr, zr, 1/zr, aa/zr + 1/zr]))
    _,l = dbscan(X, eps=0.0035, min_samples=1,)

    return l

def one_loop1(param):
    
    # <todo> tune your parameters or design your own features here! 
    
    i,m, x,y,z, d,r,r2,z2,a, a_start,a_step = param
    #print('\r %3d  %+0.8f '%(i,da), end='', flush=True)
    
    da = m*(a_start - (i*a_step))
    aa = a + np.sign(z)*z*da
#     if m == 1:
#         print(da)
    zr = z/r # this is cot(theta), 1/zr is tan(theta)
    theta = np.arctan2(r, z)
    ct = np.cos(theta)
    st = np.sin(theta)
    tt = np.tan(theta)
#     ctt = np.cot(theta)
    z2r = z2/r
    z2r2 = z2/r2
#     X = StandardScaler().fit_transform(df[['r2', 'theta_1', 'dip_angle', 'z2', 'z2_1', 'z2_2']].values)

    caa = np.cos(aa)
    saa = np.sin(aa)
    taa = np.tan(aa)
    ctaa = 1/taa
    
#     0.000005
    deps = 0.0000025
    X = StandardScaler().fit_transform(np.column_stack([caa, saa, tt, 1/tt]))
    l= DBSCAN(eps=0.0035+i*deps,min_samples=1,metric='euclidean',n_jobs=8).fit(X).labels_
#     _,l = dbscan(X, eps=0.0035, min_samples=1,algorithm='auto')
    
    return l

def one_loop2(param):
    
    # <todo> tune your parameters or design your own features here! 
    
    i,m, x,y,z, d,r,r2,z2,a, a_start,a_step = param
    #print('\r %3d  %+0.8f '%(i,da), end='', flush=True)
    
    da = m*(a_start - (i*a_step))
    aa = a + np.sign(z)*z*da
#     if m == 1:
#         print(da)
    zr = z/r # this is cot(theta), 1/zr is tan(theta)
    theta = np.arctan2(r, z)
    ct = np.cos(theta)
    st = np.sin(theta)
    tt = np.tan(theta)
#     ctt = np.cot(theta)
    z2r = z2/r
    z2r2 = z2/r2
#     X = StandardScaler().fit_transform(df[['r2', 'theta_1', 'dip_angle', 'z2', 'z2_1', 'z2_2']].values)

    caa = np.cos(aa)
    saa = np.sin(aa)
    taa = np.tan(aa)
    ctaa = 1/taa
    
#     0.000005
    deps = 0.0000025
    X = StandardScaler().fit_transform(np.column_stack([caa, saa, tt, 1/tt]))
    l= DBSCAN(eps=0.0035+i*deps,min_samples=1,metric='euclidean',n_jobs=8).fit(X).labels_
#     _,l = dbscan(X, eps=0.0035, min_samples=1,algorithm='auto')
    
    return l

def do_dbscan_predict(df):
    
    x  = df.x.values
    y  = df.y.values
    z  = df.z.values
    r  = np.sqrt(x**2+y**2)
    d  = np.sqrt(x**2+y**2+z**2)
    a  = np.arctan2(y,x)

    x2 = df['x']/d
    y2 = df['y']/d
    z2 = df['z']/r

    r2 = np.sqrt(x2**2 + y2**2)
    phi = np.arctan2(y, x)
    phi_deg= np.degrees(np.arctan2(y, x))
    phi2 = np.arctan2(y2, x2)
    phi2_deg = np.degrees(np.arctan2(y2, x2))
    
    for angle in range(-180,180,1):
    
        df1 = df.loc[(df.phi_deg>(angle-1.0)) & (df.phi_deg<(angle+1.0))]
    
        x  = df1.x.values
        y  = df1.y.values
        z  = df1.z.values
        r  = np.sqrt(x**2+y**2)
        d  = np.sqrt(x**2+y**2+z**2)
        a  = np.arctan2(y,x)

        x2 = df1['x']/d
        y2 = df1['y']/d
        z2 = df1['z']/r
    
        r2 = np.sqrt(x2**2 + y2**2)
        theta= np.arctan2(r, z)
        theta1 = np.arctan2(r2, z2)

    
        tan_dip = phi/theta
        tan_dip1 = phi/z2
        z2_1 = 1/z2
        z2_2 = phi/z2 + 1/z2

        dip_angle = np.arctan2(z2, (np.sqrt(x2**2 +y2**2)) * np.arccos(x2/np.sqrt(x2**2 + y2**2)))
        dip_angle1 = np.arctan2(z, (np.sqrt(x**2 +y**2)) * np.arccos(x2/np.sqrt(x**2 + y**2)))
        scores = []

        a_start,a_step,a_num = 0.00100,0.0000095,150
    
        params  = [(i,m, x,y,z,d,r,r2,z2, a, a_start,a_step) for i in range(a_num) for m in [-1,1]]

        if 1: 
            pool = Pool(processes=1)
            ls   = pool.map( one_loop1, params )


        if 0:
            ls = [ one_loop(param) for param in  params ]


        ##------------------------------------------------

        num_hits=len(df)
        labels = np.zeros(num_hits,np.int32)
        counts = np.zeros(num_hits,np.int32)
        for l in ls:
            c = make_counts(l)
            idx = np.where((c-counts>0) & (c<20))[0]
            labels[idx] = l[idx] + labels.max()
            counts = make_counts(labels)
       

#     cl = hdbscan.HDBSCAN(min_samples=1,min_cluster_size=7,
#                          metric='braycurtis',cluster_selection_method='leaf',algorithm='best', 
#                          leaf_size=50)
    
#     X = preprocess(df)
#     l1 = pd.Series(labels)
#     labels = np.unique(l1)
   
# #   print(X.shape)
# #   print(len(labels_org))
# #   print(len(labels_org[labels_org ==0]))
# #   print(len(labels_org[labels_org ==-1]))
    
#     n_labels = 0
#     while n_labels < len(labels):
#         n_labels = len(labels)
#         max_len = np.max(l1)
#         s = list(l1[l1 == 0].keys())
#         X = X[s]
#         print(X.shape)
#         if X.shape[0] <= 1:
#             break
#         l = cl.fit_predict(X)+max_len
# #         print(len(l))
#         l1[l1 == 0] = l
#         labels = np.unique(l1)
    
    return labels

## reference----------------------------------------------
def do_dbscan0_predict(df):
    x = df.x.values
    y = df.y.values
    z = df.z.values
    r = np.sqrt(x**2+y**2)
    d = np.sqrt(x**2+y**2+z**2)

    X = StandardScaler().fit_transform(np.column_stack([
        x/d, y/d, z/r]))
    _,labels = dbscan(X,
                eps=0.0075,
                min_samples=1,
                algorithm='auto',
                n_jobs=-1)

    #labels = hdbscan(X, min_samples=1, min_cluster_size=5, cluster_selection_method='eom')

    return labels

## reference----------------------------------------------
def do_dbscan0_predict(df):
    x = df.x.values
    y = df.y.values
    z = df.z.values
    r = np.sqrt(x**2+y**2)
    d = np.sqrt(x**2+y**2+z**2)

    X = StandardScaler().fit_transform(np.column_stack([
        x/d, y/d, z/r]))
    _,labels = dbscan(X,
                eps=0.0075,
                min_samples=1,
                algorithm='auto',
                n_jobs=-1)

    #labels = hdbscan(X, min_samples=1, min_cluster_size=5, cluster_selection_method='eom')

    return labels

def extend(submission,hits):
    df = submission.merge(hits,  on=['hit_id'], how='left')
#     df = submission.append(hits)
#     print(df.head())
    df = df.assign(d = np.sqrt( df.x**2 + df.y**2 + df.z**2 ))
    df = df.assign(r = np.sqrt( df.x**2 + df.y**2))
    df = df.assign(arctan2 = np.arctan2(df.z, df.r))

    for angle in range(-180,180,1):

        print ('\r %f'%angle, end='',flush=True)
        #df1 = df.loc[(df.arctan2>(angle-0.5)/180*np.pi) & (df.arctan2<(angle+0.5)/180*np.pi)]
        df1 = df.loc[(df.arctan2>(angle-1.0)/180*np.pi) & (df.arctan2<(angle+1.0)/180*np.pi)]

        min_num_neighbours = len(df1)
        if min_num_neighbours<4: continue

        hit_ids = df1.hit_id.values
        x,y,z = df1.as_matrix(columns=['x', 'y', 'z']).T
        r  = (x**2 + y**2)**0.5
        r  = r/1000
        a  = np.arctan2(y,x)
        tree = KDTree(np.column_stack([a,r]), metric='euclidean')

        track_ids = list(df1.track_id.unique())
        num_track_ids = len(track_ids)
        min_length=3

        for i in range(num_track_ids):
            p = track_ids[i]
            if p==0: continue

            idx = np.where(df1.track_id==p)[0]
            if len(idx)<min_length: continue

            if angle>0:
                idx = idx[np.argsort( z[idx])]
            else:
                idx = idx[np.argsort(-z[idx])]


            ## start and end points  ##
            idx0,idx1 = idx[0],idx[-1]
            a0 = a[idx0]
            a1 = a[idx1]
            r0 = r[idx0]
            r1 = r[idx1]

            da0 = a[idx[1]] - a[idx[0]]  #direction
            dr0 = r[idx[1]] - r[idx[0]]
            direction0 = np.arctan2(dr0,da0) 

            da1 = a[idx[-1]] - a[idx[-2]]
            dr1 = r[idx[-1]] - r[idx[-2]]
            direction1 = np.arctan2(dr1,da1) 


            ## extend start point
            ns = tree.query([[a0,r0]], k=min(20,min_num_neighbours), return_distance=False)
            ns = np.concatenate(ns)
            direction = np.arctan2(r0-r[ns],a0-a[ns])
            ns = ns[(r0-r[ns]>0.01) &(np.fabs(direction-direction0)<0.04)]

            for n in ns:
                df.loc[ df.hit_id==hit_ids[n],'track_id' ] = p 

            ## extend end point
            ns = tree.query([[a1,r1]], k=min(20,min_num_neighbours), return_distance=False)
            ns = np.concatenate(ns)

            direction = np.arctan2(r[ns]-r1,a[ns]-a1)
            ns = ns[(r[ns]-r1>0.01) &(np.fabs(direction-direction1)<0.04)] 

            for n in ns:
                df.loc[ df.hit_id==hit_ids[n],'track_id' ] = p
    #print ('\r')
#     df = df[['particle_id', 'weight', 'event_id', 'hit_id', 'track_id']]
    df = df[['event_id', 'hit_id', 'track_id']]
    return df

In [6]:
import hdbscan
seed = 123
np.random.seed(seed)

def shift(l, n):
    return l[n:] + l[:n]

# https://stackoverflow.com/questions/29246455/python-setting-decimal-place-range-without-rounding
def truncate(f, n):
    return math.floor(f * 10 ** n) / 10 ** n

def trackML31(df, w1, w2, w3, w4, w5, w6, w7, w8, w9, w10, w11, w12, w13, w14, w15, w16, w17, w18, Niter, 
              z_shift, theta0):
    x  = df.x.values
    y  = df.y.values
    z  = df.z.values
    
#     dz = 0
    z = z + z_shift
    
    rt  = np.sqrt(x**2+y**2)
    r  = np.sqrt(x**2+y**2+z**2)
    a0  = np.arctan2(y,x)

    x2 = x/r
    y2 = y/r
    
    
    phi = np.arctan2(y, x)
    phi_deg= np.degrees(np.arctan2(y, x))
    
    
    z1 = z/rt
    z2 = z/r
    
    z3 = np.log1p(abs(z/r))*np.sign(z)
    
    x1 = x/rt
    y1 = y/rt
    
    y3 = np.log1p(abs(y/r))*np.sign(y)
    
    theta = np.arctan2(rt, z)
    theta_deg = np.degrees(np.arctan2(rt, z))
    tt = np.tan(theta_deg)
    
    z4 = np.sqrt(abs(z/rt))
    
    x4 = np.sqrt(abs(x/r))
    y4 = np.sqrt(abs(y/r))
    
    mm = 1
    ls = []
    

#     def f(x):
#         return a0+mm*(rt+ 0.0000145*rt**2)/1000*(x/2)/180*np.pi
    
    for ii in range(Niter):
        mm = mm * (-1)

        a1 = a0+mm*(rt+ 0.0000145*rt**2)/1000*(ii/2)/180*np.pi
    
        da1 = mm*(1 + (2 * 0.0000145 * rt))/1000*(ii/2)/180*np.pi
        ia1 = a0*rt + mm*(((rt**2)/2) + (0.0000145*rt**3)/3)/1000*(ii/2)/180*np.pi
        
        saa = np.sin(a1)
        caa = np.cos(a1)
        
        raa = x*caa + y*saa
        
        t1 = theta+mm*(rt+ 0.8435*rt**2)/1000*(ii/2)/180*np.pi
        ctt = np.cos(t1)
        stt = np.sin(t1)
        
        mom2 = [truncate(np.sqrt(1 + (i**2)),4) for i in z]
        r0Inv = 2. * np.cos(a0 - theta0) / r
#         r0Inv2 = 2. * np.cos(a0 - theta0) / rt
#         r0Inv_d = -2. * np.sin(a1-t1) * da1 /r
        
        X = StandardScaler().fit_transform(np.column_stack([caa, saa, z1, z2, rt/r, x/r, y/r, z3, y1, y3, 
                                                            z4, x4, y4, raa, r0Inv, da1, ia1, mom2]))
#         print(X.shape)
        
#         X = StandardScaler().fit_transform(np.column_stack([caa,saa,z1,z2]))
        
        cx = [w1,w1,w2,w3, w4, w5, w6, w7, w8, w9, w11, w12, w13, w14, w15, w16, w17, w18]

#         cx = [w1,w1,w2,w3]
        
        X = np.multiply(X, cx)

        l= DBSCAN(eps=0.004,min_samples=1,metric='euclidean',n_jobs=4).fit(X).labels_

        ls.append(l)
    
        
    num_hits=len(df)
    labels = np.zeros(num_hits,np.int32)
    counts = np.zeros(num_hits,np.int32)
    lss = []
    for l in ls:
        c = make_counts(l)
        idx = np.where((c-counts>0) & (c<20))[0]
        labels[idx] = l[idx] + labels.max()
        counts = make_counts(labels)
    
#     lss.append(labels)
    
#     for i in range(Niter):
#         labels1 = np.zeros(num_hits,np.int32)
#         counts1 = np.zeros(num_hits,np.int32)
#         ls1 = ls.copy()
#         ls1 = shift(ls1, 1)
#         np.random.shuffle(ls1)
#         for l in ls1:
#             c = make_counts(l)
#             idx = np.where((c-counts>0) & (c<20))[0]
#             labels1[idx] = l[idx] + labels1.max()
#             counts1 = make_counts(labels1)
#         l1 = labels1.copy()
#         lss.append(l1)


#     labels = np.zeros(num_hits,np.int32)
#     counts = np.zeros(num_hits,np.int32)
    
#     for l in lss:
#         c = make_counts(l)
#         idx = np.where((c-counts>0) & (c<20))[0]
#         labels[idx] = l[idx] + labels.max()
#         counts = make_counts(labels)
    
#     df = pd.DataFrame(columns=['event_id', 'hit_id', 'track_id', 'vlm'],
#         data=np.column_stack(([int(0),]*len(hits), hits.hit_id.values, labels, hits.vlm.values))
#     )
#     df = pd.DataFrame()
#     df['hit_id']=hits.hit_id.values
#     df['vlm'] = hits.vlm.values
#     df['track_id'] = labels
    
#     for l in np.unique(labels):
#         df_l = df[df.track_id == l]
#         df_l['vlm_count'] =df_l.groupby('vlm')['vlm'].transform('count')
#         same_vlm_multiple_hits = np.any(df_l.vlm_count > 1)
#         if same_vlm_multiple_hits == True:
#             print(l)
#             which_vlm_multiple_hits = list(df_l[df_l.vlm_count > 1].index)  
#             which_vlm_multiple_hits.pop(0)
            
#             df.loc[which_vlm_multiple_hits, 'track_id'] = 9999999999
        
#     return df.track_id.values
#     sub = pd.DataFrame(columns=['event_id', 'hit_id', 'track_id'],
#         data=np.column_stack(([int(0),]*len(df), df.hit_id.values, labels))
#     )
#     sub['track_count'] = sub.groupby('track_id')['track_id'].transform('count')
# #     sub.loc[sub.track_count < 5, 'track_id'] = 0
    
#     sub1 = sub[sub.track_id <  0]
#     sub2 = sub[sub.track_id >= 0]
#     L_neg = sub1.track_id.values
#     L_pos = sub2.track_id.values
#     a = 1
#     for l in L_neg:
#         for l1 in range(a, np.iinfo(np.int32).max):
#             if l1 in L_pos:
#                 continue
#             sub.loc[sub.track_id == l, 'track_id'] = l1
#             a = l1 +1
#             break
    
#     L = list(sub.track_id.values)
#     labels = np.zeros(num_hits,np.int32)
#     for ii in range(num_hits):
#         labels[ii] = L[ii]
#     print(np.any(labels < 0))
    return labels

In [9]:
%%time
# def run_dbscan():

data_dir = '../data/train'

#     event_ids = [
#             '000001030',##
#             '000001025','000001026','000001027','000001028','000001029',
#     ]

event_ids = [
        '000001030',##

]

sum=0
sum_score=0
for i,event_id in enumerate(event_ids):
    particles = pd.read_csv(data_dir + '/event%s-particles.csv'%event_id)
    hits  = pd.read_csv(data_dir + '/event%s-hits.csv'%event_id)
    cells = pd.read_csv(data_dir + '/event%s-cells.csv'%event_id)
    truth = pd.read_csv(data_dir + '/event%s-truth.csv'%event_id)
    particles = pd.read_csv(data_dir + '/event%s-particles.csv'%event_id)
    
    truth = pd.merge(truth, particles, how='left', on='particle_id')
    hits = pd.merge(hits, truth, how='left', on='hit_id')
    

CPU times: user 1.29 s, sys: 124 ms, total: 1.41 s
Wall time: 1.7 s


In [10]:
%%time

w1 = 1.1932215111905984
w2 = 0.39740553885387364
w3 = 0.3512647720585538
w4 = 0.1470
w5 = 0.01201
w6 = 0.0003864
w7 = 0.0205
w8 = 0.0049
w9 = 0.00121
w10 = 1.4930496676654575e-05
w11 = 0.0318
w12 = 0.000435
w13 = 0.00038
w14 = 0.00072
w15 = 5.5e-05
#     w15 = 0.000265
w16 = 0.0031
w17 = 0.00021
w18 = 7.5e-05

Niter=247    
print(w18)

z_shift = 0
#     ls = []
track_id = trackML31(hits, w1,w2,w3,w4,w5,w6,w7,w8,w9,w10, w11, w12, w13, w14, w15, w16, w17, w18, Niter, 
                     z_shift)

sum_score=0
sum = 0
submission = pd.DataFrame(columns=['event_id', 'hit_id', 'track_id'],
    data=np.column_stack(([int(event_id),]*len(hits), hits.hit_id.values, track_id))
).astype(int)


for i in range(8):
    submission = extend(submission,hits)
    score = score_event(truth, submission)
    print('[%2d] score : %0.8f'%(i, score))
    sum_score += score
    sum += 1

print('--------------------------------------')
sc = sum_score/sum
print(sc)
# caa, saa: 5 mins score 0
# caa, saa, z1: 0.3942327679531816, 6 min 14s
# z1: 5.99028402551861e-05, 11 mins
# caa,saa,z1,z2: 7 mins, 0.5315668141457246

7.5e-05


TypeError: trackML31() missing 1 required positional argument: 'theta0'

In [15]:
num_hits = len(hits)
labels = np.zeros(num_hits,np.int32)
counts = np.zeros(num_hits,np.int32)
for i in range(len(ls)):
    labels1 = np.zeros(num_hits,np.int32)
    counts1 = np.zeros(num_hits,np.int32)
    ls1 = ls.copy()
    ls1 = shift(ls1, 1)
    np.random.shuffle(ls1)
    for l in ls1:
        c = make_counts(l)
        idx = np.where((c-counts>0) & (c<20))[0]
        labels1[idx] = l[idx] + labels1.max()
        counts1 = make_counts(labels1)
    l1 = labels1.copy()
    lss.append(l1)


labels = np.zeros(num_hits,np.int32)
counts = np.zeros(num_hits,np.int32)

for l in lss:
    c = make_counts(l)
    idx = np.where((c-counts>0) & (c<20))[0]
    labels[idx] = l[idx] + labels.max()
    counts = make_counts(labels)

sum_score=0
sum = 0
submission = pd.DataFrame(columns=['event_id', 'hit_id', 'track_id'],
    data=np.column_stack(([int(event_id),]*len(hits), hits.hit_id.values, labels))
).astype(int)


for i in range(8):
    submission = extend(submission,hits)
    score = score_event(truth, submission)
    print('[%2d] score : %0.8f'%(i, score))
    sum_score += score
    sum += 1

print('--------------------------------------')
sc = sum_score/sum
print(sc)

# 179.0000000[ 0] score : 0.63363358
#  179.0000000[ 1] score : 0.63765912
#  179.0000000[ 2] score : 0.63883962
#  179.0000000[ 3] score : 0.64030808
#  179.0000000[ 4] score : 0.64120567
#  179.0000000[ 5] score : 0.64168075
#  179.0000000[ 6] score : 0.64064708
#  179.0000000[ 7] score : 0.64116239
# --------------------------------------
# 0.63939203643381

 179.0000000[ 0] score : 0.63363358
 179.0000000[ 1] score : 0.63765912
 179.0000000[ 2] score : 0.63883962
 179.0000000[ 3] score : 0.64030808
 179.0000000[ 4] score : 0.64120567
 179.0000000[ 5] score : 0.64168075
 179.0000000[ 6] score : 0.64064708
 179.0000000[ 7] score : 0.64116239
--------------------------------------
0.63939203643381


In [14]:
hits.head()

,hit_id,x,y,z,volume_id,layer_id,module_id,particle_id,tx,ty,...,tpz,weight,vx,vy,vz,px,py,pz,q,nhits
0,1,-60.8722,-2.90994,-1502.5,7,2,1,427858663433043968,-60.8900,-2.93122,...,-9.71788,0.000008,0.010578,-0.000096,-2.68331,-0.396593,-0.000155,-9.72165,-1.0,12.0
1,2,-68.6209,-6.93030,-1502.5,7,2,1,923241222145835008,-68.6150,-6.94011,...,-4.86031,0.000008,0.012087,-0.023069,13.10000,-0.216069,-0.041984,-4.86522,1.0,13.0
2,3,-64.5802,2.66632,-1502.5,7,2,1,4523734434054144,-64.5634,2.67475,...,-6.74716,0.000006,-0.009604,0.008012,-2.28776,-0.292098,0.031685,-6.75259,-1.0,16.0
3,4,-88.7343,-7.35970,-1502.5,7,2,1,261225408500858880,-88.7490,-7.38212,...,-15.14960,0.000012,0.001303,0.017286,-8.23350,-0.894095,-0.099110,-15.15630,1.0,14.0
4,5,-88.3262,-13.04520,-1502.5,7,2,1,743099023757410304,-88.3163,-13.05070,...,-4.90527,0.000006,-0.011713,-0.011290,-5.07843,-0.283424,-0.068196,-4.91150,1.0,16.0


In [9]:
def Fun4BO22221(params):
    df, w1,w2,w3,w4,w5,w6,w7,w8,w9,w10, w11, w12, w13, w14, w15, w16, w17, w18, Niter, z_s, t = params
    
    l = trackML31(df, w1,w2,w3,w4,w5,w6,w7,w8,w9,w10, w11, w12, w13, w14, w15, w16, w17, w18, Niter, 
                         z_s, t)
    return l

In [10]:
%%time

def Fun4BO21(df):
    
    w1 = 1.1932215111905984
    w2 = 0.39740553885387364
    w3 = 0.3512647720585538
    w4 = 0.1470
    w5 = 0.01201
    w6 = 0.0003864
    w7 = 0.0205
    w8 = 0.0049
    w9 = 0.00121
    w10 = 1.4930496676654575e-05
    w11 = 0.0318
    w12 = 0.000435
    w13 = 0.00038
    w14 = 0.00072
    w15 = 0.00109
#     w15 = 5.5e-05

#     w15 = 1.
    w16 = 0.0031
    w17 = 0.00021
    w18 = 7.5e-05
    
    Niter=247    
#     print(w18)
   
    ls = []
    lss = []
    z_shift = 0
    
    if 1:
        params = []
        L = [0]
        L = L + list(np.linspace(-180, 180, 50))
        for t in L: 
            params.append((df, w1,w2,w3,w4,w5,w6,w7,w8,w9,w10, w11, w12, w13, w14, w15, w16, w17, w18, Niter, 
                             0, t))
            
        pool = Pool(processes=10)
        ls1 = pool.map(Fun4BO22221, params, chunksize=1)
        pool.close()
        
        j = 0
        for l in ls1:
            print(L[j])
            sum_score=0
            sum = 0
            submission = pd.DataFrame(columns=['event_id', 'hit_id', 'track_id'],
                data=np.column_stack(([int(event_id),]*len(df), df.hit_id.values, l))
            ).astype(int)


            for i in range(8):
                submission = extend(submission,df)
                score = score_event(truth, submission)
                print('[%2d] score : %0.8f'%(i, score))
                sum_score += score
                sum += 1

            print('--------------------------------------')
            sc = sum_score/sum
            print(sc)
            j += 1
            
    if 0:
        theta0 = [0]
        theta_list = list(np.linspace(-np.pi, np.pi, 50))

        theta0 = theta0 + theta_list
        params = []
        for t in theta0:
            params.append((df, w1,w2,w3,w4,w5,w6,w7,w8,w9,w10, w11, w12, w13, w14, w15, w16, w17, w18, Niter, 
                             0, t))

        pool = Pool(processes=11)
        ls1 = pool.map(Fun4BO22221, params, chunksize=1)
        pool.close()

        ls = ls + ls1
        num_hits = len(df)
    #     lss = []
        labels = np.zeros(num_hits,np.int32)
        counts = np.zeros(num_hits,np.int32)
        for i in range(len(ls)):
            labels1 = np.zeros(num_hits,np.int32)
            counts1 = np.zeros(num_hits,np.int32)
            ls1 = ls.copy()
            ls1 = shift(ls1, 1)
            np.random.shuffle(ls1)
            for l in ls1:
                c = make_counts(l)
                idx = np.where((c-counts>0) & (c<20))[0]
                labels1[idx] = l[idx] + labels1.max()
                counts1 = make_counts(labels1)
            l1 = labels1.copy()
            lss.append(l1)


        labels = np.zeros(num_hits,np.int32)
        counts = np.zeros(num_hits,np.int32)

        for l in lss:
            c = make_counts(l)
            idx = np.where((c-counts>0) & (c<20))[0]
            labels[idx] = l[idx] + labels.max()
            counts = make_counts(labels)

        sum_score=0
        sum = 0
        submission = pd.DataFrame(columns=['event_id', 'hit_id', 'track_id'],
            data=np.column_stack(([int(event_id),]*len(df), df.hit_id.values, labels))
        ).astype(int)


        for i in range(8):
            submission = extend(submission,hits)
            score = score_event(truth, submission)
            print('[%2d] score : %0.8f'%(i, score))
            sum_score += score
            sum += 1

        print('--------------------------------------')
        sc = sum_score/sum
        print(sc)
    
        return sc
    return 0
#     return labels

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 12.2 µs


In [10]:
def Fun4BO2222(params):
    df, w1,w2,w3,w4,w5,w6,w7,w8,w9,w10, w11, w12, w13, w14, w15, w16, w17, w18, Niter, z_s, t = params
    
    l = trackML31(df, w1,w2,w3,w4,w5,w6,w7,w8,w9,w10, w11, w12, w13, w14, w15, w16, w17, w18, Niter, 
                         z_s, t)
    return l

In [11]:
%%time

def Fun4BO2(df, truth):
    
    w1 = 1.1932215111905984
    w2 = 0.39740553885387364
    w3 = 0.3512647720585538
    w4 = 0.1470
    w5 = 0.01201
    w6 = 0.0003864
    w7 = 0.0205
    w8 = 0.0049
    w9 = 0.00121
    w10 = 1.4930496676654575e-05
    w11 = 0.0318
    w12 = 0.000435
    w13 = 0.00038
    w14 = 0.00072
    w15 = 0.00109
#     w15 = 5.5e-05
#     w15 = 0.000265
    w16 = 0.0031
    w17 = 0.00021
#     w18 = 7.5e-05
    w18 = 0.009 # for mom2 round(2)
    
    Niter=247    
#     print(w18)
   
    ls = []
    lss = []
    z_shift = 0

    z_shifts = [0]
    z_shift_list = list(np.linspace(-5.5, 5.5, 10))
    
    z_shifts = z_shifts + z_shift_list
    params = []
    for z_s in z_shifts:
        params.append((df, w1,w2,w3,w4,w5,w6,w7,w8,w9,w10, w11, w12, w13, w14, w15, w16, w17, w18, Niter, 
                         z_s, 0))
    
    pool = Pool(processes=len(z_shifts))
    ls1 = pool.map(Fun4BO2222, params, chunksize=1)
    pool.close()
    
    ls = ls + ls1
    num_hits = len(df)
#     lss = []
    labels = np.zeros(num_hits,np.int32)
    counts = np.zeros(num_hits,np.int32)
    for i in range(len(ls)):
        labels1 = np.zeros(num_hits,np.int32)
        counts1 = np.zeros(num_hits,np.int32)
        ls1 = ls.copy()
        ls1 = shift(ls1, 1)
        np.random.shuffle(ls1)
        for l in ls1:
            c = make_counts(l)
            idx = np.where((c-counts>0) & (c<20))[0]
            labels1[idx] = l[idx] + labels1.max()
            counts1 = make_counts(labels1)
        l1 = labels1.copy()
        lss.append(l1)


    labels = np.zeros(num_hits,np.int32)
    counts = np.zeros(num_hits,np.int32)
    
    for l in lss:
        c = make_counts(l)
        idx = np.where((c-counts>0) & (c<20))[0]
        labels[idx] = l[idx] + labels.max()
        counts = make_counts(labels)
        
    sum_score=0
    sum = 0
    submission = pd.DataFrame(columns=['event_id', 'hit_id', 'track_id'],
        data=np.column_stack(([int(event_id),]*len(df), df.hit_id.values, labels))
    ).astype(int)
    
    
    for i in range(8):
        submission = extend(submission,df)
        score = score_event(truth, submission)
        print('[%2d] score : %0.8f'%(i, score))
        sum_score += score
        sum += 1

    print('--------------------------------------')
    sc = sum_score/sum
    print(sc)
    
    return sc
#     return labels

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 15.3 µs


In [ ]:
%%time
# def run_dbscan():

data_dir = '../data/train'

#     event_ids = [
#             '000001030',##
#             '000001025','000001026','000001027','000001028','000001029',
#     ]

event_ids = [
        '000001030',##

]

sum=0
sum_score=0
for i,event_id in enumerate(event_ids):
    particles = pd.read_csv(data_dir + '/event%s-particles.csv'%event_id)
    hits  = pd.read_csv(data_dir + '/event%s-hits.csv'%event_id)
    cells = pd.read_csv(data_dir + '/event%s-cells.csv'%event_id)
    truth = pd.read_csv(data_dir + '/event%s-truth.csv'%event_id)
    particles = pd.read_csv(data_dir + '/event%s-particles.csv'%event_id)
    
    truth = pd.merge(truth, particles, how='left', on='particle_id')
    hits = pd.merge(hits, truth, how='left', on='hit_id')
    
#     bo = BayesianOptimization(Fun4BO,pbounds = {'w1':w1,'w2':w2,'w3':w3,'Niter':Niter})
#     bo.maximize(init_points = 3, n_iter = 20, acq = "ucb", kappa = 2.576)
#     w1 = 1.1932215111905984
#     w2 = 0.39740553885387364
#     w3 = 0.3512647720585538
#     w4 = [0.1, 0.2] # 0.1470 -> 0.55690
#     w4 = 0.1470
#     w5 = [0.001, 1.2] # 0.7781 -> 0.55646, 0.7235 + N = 247 => 0.56025
    
#     Niter = 179
#     Niter = 247
#     w5 = 0.01
#     for w6 in [0.012, 0.01201, 0.01202, 0.01203, 0.01204, 0.01205, 0.01206, 0.01207, 0.01208, 0.01209, 0.0121]:
#     EPS = 1e-12
#     w6 = [0.001, 1.2]
#     w6 = 0.0205
#     w18 = [0.00001, 0.05]
#     w13 = 0.00038
#     w14 = 0.0007133505234834969
#     for w8 in np.arange(0.00008, 0.00015, 0.000005):
#         print(w8)
#     Fun4BO2(1)
#     for w18 in np.arange(1.0e-05, 9.0e-05, 5.0e-06):
#         print(w18)
    Fun4BO2(hits, truth)
#     Niter = [240, 480]
#     w18 = [0.00001, 0.0003]
#     bo = BayesianOptimization(Fun4BO2,pbounds = {'w18':w18})
#     bo.maximize(init_points = 20, n_iter = 5, acq = "ucb", kappa = 2.576)

# x/y: 7 | 06m30s |    0.55302 |    0.0100 | 
# x/y: 0.001: 0.55949
# x/y: 0.0001: 0.55949
# x/y: 0.002: 0.55959
# x/y: 0.003: 0.55915
# x/y: 0.0025: 0.55925
# x/y: 0.0015: 0.55953
# x/r: 0.0015: 0.56186
# x/r: 0.002: 0.56334
# x/r: 0.0025: 0.563989
# x/r: 0.003: 0.56447
# x/r: 0.01: 0.569822
# x/r: 0.015: 0.56940
# x/r: 0.012: 0.5719
# x/r: 0.01201: 0.57192
# 1.4499999999999993e-05 * rt**2: 0.5720702851970194
# 0.0000145
# z3: 10 | 07m12s |    0.57208 |    0.0205 | 
# count: 19: 0.572567, 17: 0.57263
# ctt, stt after change:  2 | 07m56s |    0.57345 |    0.0001 | (0.00010567777727496665)
# x4: 25 | 09m42s |    0.57359 |    0.0002 | (0.000206214286412982)
# x4: 0.000435 (0.5737387485278771) (x4 = np.sqrt(abs(x/r)))
# w13: 00038 (ctt,stt): 0.5737528800479372
# ensemble of 10: 0.5772859116242378
# ensemble of Niter=247 (random shuffle+ shift): 0.5787580886742594
# ensemble of Niter=247 (shift only): 0.5743461440542145
# ensemble of Niter=247 (random shuffle+ shift+ eps=0.004+vlm): 0.5865991424251623
# 14 + ensemble: (0.0007133505234834969) 0.58787
# w14 + ensemble: 1 | 30m13s |    0.58787 |    0.0007 | (0.0007133505234834969)
# w14: 0.00027 (0.5873896523922799)
# test w14, raa = x*caa + y*saa(0.00072: 0.5878990304956998)
# test w16: r0Inv1 (21 | 21m40s |    0.58735 |    0.0000 | (1.0002801729384074e-05))
# test w16: r0Inv1 (5.5e-06: 0.5881860039044223)
# test r0Inv1 (5.5e-06, Niter=246, 0.5867403075395137)
# test r0Inv1 (5.5e-06, Niter=247, 0.5872846547180826)
# Niter = 247 (0.5880986018552999):
# X = StandardScaler().fit_transform(np.column_stack([caa, saa, z1, z2, rt/r, x/r, y/r, z3, y1, y3, 
#                                                             ctt, stt, z4, x4, y4, raa, r0Inv]))
#         cx = [w1,w1,w2,w3, w4, w5, w6, w7, w8, w9, w10, w10, w11, w12, w13, w14, w15]
# w15: 5.0611615056082495e-05 (17 | 21m37s |    0.58790 |    0.0001 | )
# w15 test (5.5e-05: 0.5881768870518835)
# w15 alone: 5.5e-05: 0.5870504337495849
# w15 again: 5.5e-05 0.5864220587506578 (strange)
# w15 again: 5.5e-05 (0.5880689577051738)
# w16: 5.5e-06: 0.587602145623185 (bad since w16 was not being used)
# after reset: w16 not being used - 0.5880689577051738
# a2: 0.0206: 0.58157
# org (no shift + ensemble): 0.5859135274547416
# org (with shift + ensemble + ia1): 0.5901965251117371
# org (with shift + ensemble + no ia1): 0.5901656684266057
# r0Inv_d1: 7.401866174854672e-05, 0.58592 (7.5e-05: 0.5892)
# multiprocessing - 0.6377253549867099 ( 25 mins: i7)
# multiprocessing -  z_shift (10 linspace + 1)- 0.637725354987 (32  mins)
# z_shift (20 linspace + 1) - 0.639556619038 (44 mins)
# z_shift ( 5 linspace + 1) - (0.62971260568) (19 mins)
# theta0 - w15= 1.0, 1 hr 40  mins, 0.397164972826)
# theta:0 , r0Inv using r, w15: (0.1: 0.263406003077, 0.01: 0.555401034484, 0.001:0.58657067095, 0.0001: 0.585995954153, 
# 1e-05: 0.585960618134 )
# w15: 0.0009: 0.586267891213, 0.015: 0.586183567052, 0.002: 0.585828179853)
# theta:0 , r0Inv using rt, w15: (0.1: 0.145904897094, 0.01: 0.536585229252, 0.001:0.585387197245, 0.0001: 0.585950889554, 
# 1e-05 to 7e-05: 0.585960618134, 40 min, 8e-05, 9e-05: 0.585950889554 )
# theta: 0, r0Inv using r, 0.00109:0.58714214929 
# theta: 50: 0.586437243616 (in radians) (Wall time: 2h 26min 56s)
# mom2 with z_shift round 1

In [2]:
cluster_runs = np.random.randint(0, 50, (50, 15000))

In [3]:
cluster_runs.shape

(50, 15000)

In [3]:
consensus_clustering_labels = CE.cluster_ensembles(cluster_runs, verbose = True, N_clusters_max = 50)


INFO: Cluster_Ensembles: cluster_ensembles: due to a rather large number of cells in your data-set, using only 'HyperGraph Partitioning Algorithm' (HGPA) and 'Meta-CLustering Algorithm' (MCLA) as ensemble consensus functions.


*****
INFO: Cluster_Ensembles: HGPA: consensus clustering using HGPA.

#
INFO: Cluster_Ensembles: wgraph: writing wgraph_HGPA.
INFO: Cluster_Ensembles: wgraph: 15000 vertices and 2500 non-zero hyper-edges.
#

#
INFO: Cluster_Ensembles: sgraph: calling shmetis for hypergraph partitioning.
INFO: Cluster_Ensembles: sgraph: (hyper)-graph partitioning completed; loading wgraph_HGPA.part.50
#

INFO: Cluster_Ensembles: cluster_ensembles: HGPA at 0.02434736018512536.
*****

*****
INFO: Cluster_Ensembles: MCLA: consensus clustering using MCLA.
INFO: Cluster_Ensembles: MCLA: preparing graph for meta-clustering.
INFO: Cluster_Ensembles: MCLA: done filling hypergraph adjacency matrix. Starting computation of Jaccard similarity matrix.
INFO: Cluster_Ensembles: MCLA: startin

In [4]:
cluster_runs.shape

(50, 15000)

In [5]:
consensus_clustering_labels.shape

(15000,)

In [11]:
w1 = 1.1932215111905984
w2 = 0.39740553885387364
w3 = 0.3512647720585538
w4 = 0.1470
w5 = 0.01201
w6 = 0.0003864
w7 = 0.0205
w8 = 0.0049
w9 = 0.00121
w10 = 1.4930496676654575e-05
w11 = 0.0318
w12 = 0.000435
w13 = 0.00038
w14 = 0.00072
w15 = 5.5e-05
#     w15 = 0.000265
w16 = 0.0031
w17 = 0.00021
w18 = 7.5e-05

Niter=247    

In [12]:
def run_make_submission():

    data_dir = '../data/test'


    tic = t = time.time()
    event_ids = [ '%09d'%i for i in range(0,125) ]  #(0,125)

    if 1:
        submissions = []
        for i,event_id in enumerate(event_ids):
            hits  = pd.read_csv(data_dir + '/event%s-hits.csv'%event_id)
            cells = pd.read_csv(data_dir + '/event%s-cells.csv'%event_id)
            
            labels = Fun4BO2(hits)

            toc =  time.time()
            print('\revent_id : %s  , %0.0f min'%(event_id, (toc-tic)/60))

            # Prepare submission for an event
            submission = pd.DataFrame(columns=['event_id', 'hit_id', 'track_id'],
                data=np.column_stack(([event_id,]*len(hits), hits.hit_id.values, labels))
            ).astype(int)
            submissions.append(submission)
            
            for i in range(8):
                submission = extend(submission,hits)
            
            
            submission.to_csv('../cache/sub2/%s.csv.gz'%event_id,
                                index=False, compression='gzip')

            #------------------------------------------------------
    if 1:

        event_ids = [ '%09d'%i for i in range(0,125) ]  #(0,125)
        submissions = []
        for i,event_id in enumerate(event_ids):
            submission  = pd.read_csv('../cache/sub2/%s.csv.gz'%event_id, compression='gzip')
            submissions.append(submission)

        
        # Create submission file
        submission = pd.concat(submissions, axis=0)
        submission.to_csv('../submissions/sub2/submission-0029.csv.gz',
                            index=False, compression='gzip')
        print(len(submission))

In [ ]:
run_make_submission()

event_id : 000000000  , 22 min
event_id : 000000001  , 48 min
event_id : 000000002  , 70 min
 179.0000000